###  Generate a moons dataset using make_moons(n_samples=10000, noise=0.4).

In [1]:
from sklearn.datasets import make_moons

moons_dataset = make_moons(n_samples=10000, noise=0.4)
X,y = moons_dataset

### Split it into a training set and a test set using train_test_split().

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state=42)

In [3]:
X.shape

(10000, 2)

In [4]:
y.shape

(10000,)

### Use grid search with cross-validation (with the help of the GridSearchCV class) to find good hyperparameter values for a DecisionTreeClassifier. Hint: try various values for max_leaf_nodes.

In [5]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train,y_train)

DecisionTreeClassifier()

In [6]:
# Predict using test set

y_pred = tree_clf.predict(X_test)

In [7]:
# Checking accuracy score of prediction

from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.8013333333333333

In [8]:
# checking confusin matrix

from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[1200,  295],
       [ 301, 1204]])

In [9]:
from sklearn.model_selection import GridSearchCV

params = [{
    'min_samples_split' : [2,3,4],
    'max_leaf_nodes' : list(range(2,100))
}]

In [10]:
grid_search = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    params,
    verbose=1,
    cv=3
)

In [11]:
# fit data into grid search

grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 882 out of 882 | elapsed:    7.4s finished


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid=[{'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                             13, 14, 15, 16, 17, 18, 19, 20, 21,
                                             22, 23, 24, 25, 26, 27, 28, 29, 30,
                                             31, ...],
                          'min_samples_split': [2, 3, 4]}],
             verbose=1)

In [12]:
grid_search.best_estimator_  #checking for best estimataor of our classifier

DecisionTreeClassifier(max_leaf_nodes=9, random_state=42)

### Train it on the full training set using these hyperparameters, and measure your model’s performance on the test set. You should get roughly 85% to 87% accuracy.

In [17]:
clf = DecisionTreeClassifier(max_leaf_nodes=9, random_state=42)

In [18]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_leaf_nodes=9, random_state=42)

In [19]:
yy_pred = clf.predict(X_test)

In [25]:
accuracy_score(y_test, yy_pred) # checking accuracy score

0.8593333333333333

In [24]:
from sklearn.tree import export_graphviz

export_graphviz(
    clf,
    out_file=('moons_tree.dot'),
    feature_names=None,
    class_names=None,
    filled=True,
)

### Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit- Learn’s ShuffleSplit class for this.

In [26]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

### Train one Decision Tree on each subset, using the best hyperparameter values found above. Evaluate these 1,000 Decision Trees on the test set. Since they were trained on smaller sets, these Decision Trees will likely perform worse than the first Decision Tree, achieving only about 80% accuracy.


In [28]:
clf.fit(X_mini_train, y_mini_train)

DecisionTreeClassifier(max_leaf_nodes=9, random_state=42)

In [29]:
ypred = clf.predict(X_test)

In [30]:
accuracy_score(y_test, ypred)

0.782

### Now comes the magic. For each test set instance, generate the predictions of the 1,000 Decision Trees, and keep only the most frequent prediction (you can use SciPy’s mode() function for this). This gives you majority-vote predictions over the test set.